In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, when, upper

In [2]:
spark = SparkSession.builder.appName("ColExampleApp").getOrCreate()

In [4]:
df1 = spark.createDataFrame([(1, "Alice"), (2, "Bob")], ["id", "value"]).alias("t1")
df2 = spark.createDataFrame([(1, "X"), (2, "Y")], ["id", "value"]).alias("t2")

joined = df1.join(df2, df1["id"] == df2["id"])
joined.show()

result = joined.select(col("t2.value"))
result.show()

+---+-----+---+-----+
| id|value| id|value|
+---+-----+---+-----+
|  1|Alice|  1|    X|
|  2|  Bob|  2|    Y|
+---+-----+---+-----+

+-----+
|value|
+-----+
|    X|
|    Y|
+-----+



In [6]:
data = [(1, "Alice", 20), (2, "Bob", 11), (3, "Charlie", 30)]
columns = ["id", "name", "age"]

if_expr = when(df3['age'] < 18, 'underage').otherwise('adult')
df3 = spark.createDataFrame(data, columns).withColumn('age_category', if_expr)
df3.show()

AnalysisException: [MISSING_ATTRIBUTES.RESOLVED_ATTRIBUTE_APPEAR_IN_OPERATION] Resolved attribute(s) "age" missing from "id", "name", "age" in operator !Project [id#106L, name#107, age#108L, CASE WHEN (age#80L < cast(18 as bigint)) THEN underage ELSE adult END AS age_category#112]. Attribute(s) with the same name appear in the operation: "age".
Please check if the right attribute(s) are used.;
!Project [id#106L, name#107, age#108L, CASE WHEN (age#80L < cast(18 as bigint)) THEN underage ELSE adult END AS age_category#112]
+- LogicalRDD [id#106L, name#107, age#108L], false


In [9]:
result = joined.withColumn("flag", when(col("t1.id") > 1, 1).otherwise(0)).filter(col("flag") > 0)

result.show()

+---+-----+---+-----+----+
| id|value| id|value|flag|
+---+-----+---+-----+----+
|  2|  Bob|  2|    Y|   1|
+---+-----+---+-----+----+



In [6]:
df3.withColumn("name", upper(df3["name"])).filter(df3["name"] == "ALICE").toPandas()

,id,name,age,age_category


In [7]:
df3.withColumn("name", upper(df3["name"])).filter(col("name") == "ALICE").toPandas()

,id,name,age,age_category
0,1,ALICE,20,adult


In [10]:
df4 = df3.withColumn("name", upper(df3["name"]))

In [12]:
df3.explain()

== Physical Plan ==
*(1) Project [id#78L, name#79, age#80L, CASE WHEN (age#80L < 18) THEN underage ELSE adult END AS age_category#84]
+- *(1) Scan ExistingRDD[id#78L,name#79,age#80L]




In [13]:
df4.explain()

== Physical Plan ==
*(1) Project [id#78L, upper(name#79) AS name#173, age#80L, CASE WHEN (age#80L < 18) THEN underage ELSE adult END AS age_category#84]
+- *(1) Scan ExistingRDD[id#78L,name#79,age#80L]


